Course: **Introduction to Artificial Intelligence** \
Lecturer: **Nguyen Thanh An** \
Lab 03: **Local Search**

Students implement Hill-Climbing Search, Local Beam Search, and Simulated Annealing Search algorithms following TODO 1 - 3. \
Students can add supporting attributes and methods to the three classes as needed.

# Libraries

In [35]:
import os
import heapq 
import math
import random


# Graph class

In [4]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0
    self.H = dict()

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    for u, h in self.H.items():
      line = 'h(%d) = %d\n'%(u, h)
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...
            u1 h1
            u2 h2
            u3 h3
            ...

        # input.txt
         7 8
        0 1 5 
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
        0 14
        1 13
        2 12
        3 11
        4 10
        5 9
        6 0
 
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for i in range(self.E):
          line = g.readline()
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))
        for i in range(self.V):
          line = g.readline()
          u, h = [int(it) for it in line.strip().split()]
          self.H[u] = h

In [5]:
g = Graph()
g.load_from_file('input.txt')
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]



# Search Strategies

In [6]:
class LocalSearchStrategy:
  def search(self, g: Graph, src: int) -> tuple:
    visited = set()
    pq = [(g.H[src], src)] # priority queue with heuristic value as priority
    while pq:
      h_u, u = heappop(pq)
      if u in visited:
        continue
      visited.add(u)
      if h_u == 0:
        return (u, g.H[u]) # found local maximum
      for v, w in g.AL[u]:
        if v not in visited:
          heappush(pq, (g.H[v], v))
    return (src, g.H[src]) # no improvement found, return source vertex

In [31]:
class HillClimbingSearch(LocalSearchStrategy):
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u

    Note: weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    '''
    # Set the initial state to the source node
    current_node = src
    # Set the initial priority/weight/desirability/cost to the heuristic value of the source node
    current_priority = g.H[src]

    # Continue searching until a local maximum is found
    while True:
      # Find the neighbors of the current node
      neighbors = g.AL.get(current_node, [])
      
      # If there are no neighbors, we've reached a local maximum
      if not neighbors:
        break
      
      # Find the neighbor with the highest priority/weight/desirability/cost
      next_node, next_priority = max(neighbors, key=lambda x: g.H[x[0]])

      # If the next node has a lower priority/weight/desirability/cost than the current node, we've reached a local maximum
      if next_priority < current_priority:
        break
      
      # Update the current node and priority/weight/desirability/cost to the next node and priority/weight/desirability/cost
      current_node = next_node
      current_priority = next_priority

    return (current_node, current_priority)

In [32]:
class LocalBeamSearch(LocalSearchStrategy):
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u

    Note:
    - weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    - parameter n is provided in the constructor
    '''
    return (None, None)

In [39]:
class SimulatedAnnealingSearch(LocalSearchStrategy):
    def search(self, g: Graph, src: int) -> tuple:
        '''
        return a tuple (u, p) in which
          u: the local maximum state
          p: the priority/weight/desirability/cost of u

        Note: schedule(t) = 1/(t^2) with t is the iteration step
        '''
        current_node = src
        current_value = g.H[src]

        for t in range(1, 10001):  # set maximum number of iterations to 10000
            T = 1.0 / (t**2)   # calculate temperature
            neighbor_node = None
            neighbor_value = None

            # find a random neighbor node
            while not neighbor_node:
                v, w = random.choice(g.AL[current_node])
                if v != current_node:
                    neighbor_node = v
                    neighbor_value = g.H[neighbor_node]

            # calculate the change in value (delta)
            delta = neighbor_value - current_value

            # if neighbor_node is better than current_node, accept it
            if delta > 0:
                current_node = neighbor_node
                current_value = neighbor_value

            # if neighbor_node is worse than current_node, accept it with probability p
            else:
                p = math.exp(delta/T)
                if random.random() < p:
                    current_node = neighbor_node
                    current_value = neighbor_value

        # return the current node as the local maximum state
        return (current_node, current_value)


# Evaluation

In [40]:
hcs = HillClimbingSearch()
lbs = LocalBeamSearch()
sas = SimulatedAnnealingSearch()

for stg in [hcs, lbs, sas]:
  print(stg)
  u, p = stg.search(g, 0)
  print(u, p)

0 14
None None
0 14


# Submission Notice


*   Maintain all cell outputs
*   Download and rename the notebook as **lab03_\<Student ID\>.ipynb**
*   Submit by the deadline
